# Машинное обучение, ФКН ВШЭ

## Практическое задание 8. Метод опорных векторов и аппроксимация ядер

### Общая информация
Дата выдачи: 05.02.2021

Мягкий дедлайн: 01:59MSK 21.02.2021

Жесткий дедлайн: 01:59MSK 24.02.2021

### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимальная оценка за работу (без учёта бонусов) — 10 баллов.

Сдавать задание после указанного жёсткого срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.

### Формат сдачи
Задания сдаются через систему anytask. Посылка должна содержать:
* Ноутбук homework-practice-08-random-features-Username.ipynb

Username — ваша фамилия и имя на латинице именно в таком порядке

### О задании

На занятиях мы подробно обсуждали метод опорных векторов (SVM). В базовой версии в нём нет чего-то особенного — мы всего лишь используем специальную функцию потерь, которая не требует устремлять отступы к бесконечности; ей достаточно, чтобы отступы были не меньше +1. Затем мы узнали, что SVM можно переписать в двойственном виде, который, позволяет заменить скалярные произведения объектов на ядра. Это будет соответствовать построению модели в новом пространстве более высокой размерности, координаты которого представляют собой нелинейные модификации исходных признаков.

Ядровой SVM, к сожалению, довольно затратен по памяти (нужно хранить матрицу Грама размера $d \times d$) и по времени (нужно решать задачу условной оптимизации с квадратичной функцией, а это не очень быстро). Мы обсуждали, что есть способы посчитать новые признаки $\tilde \varphi(x)$ на основе исходных так, что скалярные произведения этих новых $\langle \tilde \varphi(x), \tilde \varphi(z) \rangle$ приближают ядро $K(x, z)$.

Мы будем исследовать аппроксимации методом Random Fourier Features (RFF, также в литературе встречается название Random Kitchen Sinks) для гауссовых ядер. Будем использовать формулы, которые немного отличаются от того, что было на лекциях (мы добавим сдвиги внутрь тригонометрических функций и будем использовать только косинусы, потому что с нужным сдвигом косинус превратится в синус):
$$\tilde \varphi(x) = (
\cos (w_1^T x + b_1),
\dots,
\cos (w_n^T x + b_n)
),$$
где $w_j \sim \mathcal{N}(0, 1/\sigma^2)$, $b_j \sim U[-\pi, \pi]$.

На новых признаках $\tilde \varphi(x)$ мы будем строить любую линейную модель.

Можно считать, что это некоторая новая парадигма построения сложных моделей. Можно направленно искать сложные нелинейные закономерности в данных с помощью градиентного бустинга или нейронных сетей, а можно просто нагенерировать большое количество случайных нелинейных признаков и надеяться, что быстрая и простая модель (то есть линейная) сможет показать на них хорошее качество. В этом задании мы изучим, насколько работоспособна такая идея.

### Алгоритм

Вам потребуется реализовать следующий алгоритм:
1. Понизить размерность выборки до new_dim с помощью метода главных компонент.
2. Для полученной выборки оценить гиперпараметр $\sigma^2$ с помощью эвристики (рекомендуем считать медиану не по всем парам объектов, а по случайному подмножеству из где-то миллиона пар объектов): $$\sigma^2 = \text{median}_{i, j = 1, \dots, \ell, i \neq j} \left\{\sum_{k = 1}^{d} (x_{ik} - x_{jk})^2 \right\}$$
3. Сгенерировать n_features наборов весов $w_j$ и сдвигов $b_j$.
4. Сформировать n_features новых признаков по формулам, приведённым выше.
5. Обучить линейную модель (логистическую регрессию или SVM) на новых признаках.
6. Повторить преобразования (PCA, формирование новых признаков) к тестовой выборке и применить модель.

Тестировать алгоритм мы будем на данных Fashion MNIST. Ниже код для их загрузки и подготовки.

In [1]:
import keras
from keras.datasets import fashion_mnist
(x_train_pics, y_train), (x_test_pics, y_test) = fashion_mnist.load_data()
x_train = x_train_pics.reshape(x_train_pics.shape[0], -1)
x_test = x_test_pics.reshape(x_test_pics.shape[0], -1)

__Задание 1. (5 баллов)__

Реализуйте алгоритм, описанный выше. Можете воспользоваться шаблоном класса ниже или написать свой интерфейс.

Ваша реализация должна поддерживать следующие опции:
1. Возможность задавать значения гиперпараметров new_dim (по умолчанию 50) и n_features (по умолчанию 1000).
2. Возможность включать или выключать предварительное понижение размерности с помощью метода главных компонент.
3. Возможность выбирать тип линейной модели (логистическая регрессия или SVM с линейным ядром).

Протестируйте на данных Fashion MNIST, сформированных кодом выше. Если на тесте у вас получилась доля верных ответов не ниже 0.84 с гиперпараметрами по умолчанию, то вы всё сделали правильно.

In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

class RFFPipeline(BaseEstimator, TransformerMixin):
    def __init__(self, n_features=1000, new_dim=50, use_PCA=True, classifier='logreg'):
        """        
        Implements pipeline, which consists of PCA decomposition,
        Random Fourier Features approximation and linear classification model.
        
        n_features, int: amount of synthetic random features generated with RFF approximation.

        new_dim, int: PCA output size.
        
        use_PCA, bool: whether to include PCA preprocessing.
        
        classifier, string: either 'svm' or 'logreg', a linear classification model to use on top of pipeline.
        
        Feel free to edit this template for your preferences.    
        """
        self.n_features = n_features
        self.use_PCA = use_PCA
        self.new_dim = new_dim
        self.classifier = classifier
        
        self.X = None
        self.y = None
        
        self.w = None
        self.b = None
        self.phi_wave = None
        
        self.model = None
        
    def fit(self, X, y):
        """
        Fit all parts of algorithm (PCA, RFF, Classification) to training set.
        """
        ## Реализация PCA
        if self.use_PCA == True:
            from sklearn.decomposition import PCA
            pca = PCA(n_components = self.new_dim)
            self.X = pca.fit_transform(X)
        else:
            self.X = X
            
        self.y = y
        
        ## Оценка вариации
        
        ind11 = np.random.randint(0, self.X.shape[0], size = 1000000)
        ind21 = np.random.randint(0, self.X.shape[0], size = 1000000)
        
        ind12 = ind11[ind11!=ind21]
        ind22 = ind21[ind21!=ind11]
        
        sigma2 = np.median(np.sum((x_train[ind12] - x_train[ind22])**2, axis = 1))
        
        ## Генерация весов и сдвигов
        
        w = np.random.normal(0, scale = 1/np.sqrt(sigma2), size = (self.new_dim, self.n_features))
        b = np.random.uniform(-np.pi, np.pi, size = (1, self.n_features))
        
        ## Генерация признаков phi_wave
        
        phi_wave = np.cos(self.X@w + np.repeat(b, self.X.shape[0], axis = 0))
        
        ## Присвоение значений классу
        
        self.w = w
        self.b = b
        self.phi_wave = phi_wave
        
        ## Обучение
        
        if self.classifier == 'logreg':
            from sklearn.linear_model import LogisticRegression
            m = LogisticRegression()
            m.fit(self.phi_wave, self.y)
            self.model = m
        else:
            from sklearn.svm import SVC
            m = SVC(kernel = 'linear')
            m.fit(self.phi_wave, self.y)
            self.model = m
        
        return self

    def predict_proba(self, X):
        """
        Apply pipeline to obtain scores for input data.
        """
        if self.classifier == 'logreg':
            return self.model.predict_proba(X)
        else:
            return self.model.predict(X) ## Подумать че б с этим сделать
            
        #raise NotImplementedError
        
    def predict(self, X):
        """
        Apply pipeline to obtain discrete predictions for input data.
        """
        return self.model.predict(X)
        #raise NotImplementedError

In [14]:
m = RFFPipeline()
m.fit(x_train, y_train)

RFFPipeline()

In [10]:
x_test_new.shape

(10000, 1000)

In [20]:
a = x_new[0:4]

In [22]:
np.cos((a @ (np.repeat(m.w, a.shape[1], axis = 1).T)) + np.repeat(m.b, a.shape[0], axis = 0))

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 50000 is different from 50)

In [17]:
upca = True
new_dim = 50

if upca == True:
    from sklearn.decomposition import PCA
    pca = PCA(n_components = new_dim)
    x_new = pca.fit_transform(x_test)
    x_test_new = x_new@m.w + np.repeat(m.b, x_new.shape[0], axis = 0)
    preds = m.predict(x_test_new)

In [18]:
np.sum(preds==y_test)/preds.shape[0]

0.1214

In [23]:
x_new@m.w + np.repeat(m.b, x_new.shape[0], axis = 0)

array([[ -2.47923788,   0.10968286, -23.23398105, ...,  -2.58801828,
          0.79798227,   2.25770143],
       [ -2.20763884,  30.06267849,   7.10691505, ...,  -6.57938466,
          5.12466898,  18.75299299],
       [ 11.96288388,   3.001315  ,  25.38608846, ...,   0.96300006,
          2.3406498 , -19.43063661],
       ...,
       [ -6.43083906,  -3.55989978,   2.1021756 , ...,  -0.71414711,
         -5.21791521,  -5.58542013],
       [  8.08759881,  -7.92444581,   6.37150835, ...,  -5.82849336,
        -14.62354965, -17.71737925],
       [ -2.53495477,  -3.02171618, -12.52211261, ...,  -1.05476404,
          3.2766452 ,   1.00892712]])

__Задание 2. (3 балла)__

Сравните подход со случайными признаками с обучением SVM на исходных признаках. Попробуйте вариант с обычным (линейным) SVM и с ядровым SVM. Ядровой SVM может очень долго обучаться, поэтому можно делать любые разумные вещи для ускорения: брать подмножество объектов из обучающей выборки, например.

Сравните подход со случайными признаками с вариантом, в котором вы понижаете размерность с помощью PCA и обучаете градиентный бустинг. Используйте одну из реализаций CatBoost/LightGBM/XGBoost, не забудьте подобрать число деревьев и длину шага.

Сделайте выводы — насколько идея со случайными признаками работает? Сравните как с точки зрения качества, так и с точки зрения скорости обучения и применения.

In [ ]:
## Обучение SVM на случайных признаках (линейная)

In [ ]:
## Обучение SVM на исходных признаках (линейная)

In [ ]:
## Обучение пакетного SVM на случайных признаках (линейная)
from sklearn.svm import SVC
from datetime import datetime

## Возьмем параметры из первой задачи
new_dim = 50
n_features = 1000

## Сгенерируем новые признаки

pca = PCA(n_components = new_dim)
x_train_new = pca.fit_transform(x_train)
pca = PCA(n_components = new_dim)
x_test_new = pca.fit_transform(x_test)

w = np.random.normal(0, scale = 1/np.sqrt(sigma2), size = (self.new_dim, self.n_features))
b = np.random.uniform(-np.pi, np.pi, size = (1, self.n_features))

phi_wave_train = np.cos(x_train_new@w + np.repeat(b, x_train_new.shape[0], axis = 0))
phi_wave_test = np.cos(x_test_new@w + np.repeat(b, x_test_new.shape[0], axis = 0))

## Обучим на них модель

lsvc = SVC(kernel = 'linear')

now = datetime.now()

lsvc.fit(phi_wave_train, y_train)

t = datetime.now() - now

print(f'Было затрачено времени: {t}')

preds = lsvc.predict(phi_wave_test)

print(f'Accuracy score: {accuracy_score(preds, y_test)}')

In [ ]:
## Обучение пакетного SVM на исходных признаках (линейная), понижение размерности не требуется
from sklearn.svm import SVC
from datetime import datetime

## Возьмем параметры из первой задачи
new_dim = 50
n_features = 1000

## Обучаем на исходных признаках

lsvc = SVC(kernel = 'linear')

now = datetime.now()

lsvc.fit(x_train, y_train)

t = datetime.now() - now

print(f'Было затрачено времени: {t}')

preds = lsvc.predict(x_test)

print(f'Accuracy score: {accuracy_score(preds, y_test)}')

In [ ]:
## Обучение пакетного SVM на случайных признаках (ядровая)

In [ ]:
## Обучение пакетного SVM на исходных признаках (ядровая)

In [ ]:
## Градиентный бустинг с понижением размерности

from catboost import CatBoostClassifier
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

pca = PCA(n_components = new_dim)
x_train_new = pca.fit_transform(x_train)
pca = PCA(n_components = new_dim)
x_test_new = pca.fit_transform(x_test)

parameters = {'iterations':[1, 10, 100, 150, 200], 'learning_rate':[0.01, 0.05, 0.3, 0.5, 0.6]}
cb_model = CatBoostClassifier()
gs = GridSearchCV(cb_model, parameters)
gs.fit(x_train_new, y_train)

0:	learn: 2.2668782	total: 140ms	remaining: 0us
0:	learn: 2.2673428	total: 105ms	remaining: 0us
0:	learn: 2.2665968	total: 136ms	remaining: 0us
0:	learn: 2.2665967	total: 123ms	remaining: 0us
0:	learn: 2.2673397	total: 154ms	remaining: 0us
0:	learn: 2.1279609	total: 101ms	remaining: 0us
0:	learn: 2.1302473	total: 151ms	remaining: 0us
0:	learn: 2.1265859	total: 114ms	remaining: 0us
0:	learn: 2.1265849	total: 138ms	remaining: 0us
0:	learn: 2.1302332	total: 140ms	remaining: 0us
0:	learn: 1.4787580	total: 95.4ms	remaining: 0us
0:	learn: 1.4920959	total: 112ms	remaining: 0us
0:	learn: 1.4728093	total: 129ms	remaining: 0us
0:	learn: 1.4725994	total: 149ms	remaining: 0us
0:	learn: 1.4921943	total: 130ms	remaining: 0us
0:	learn: 1.2903277	total: 131ms	remaining: 0us
0:	learn: 1.3099231	total: 121ms	remaining: 0us
0:	learn: 1.2840488	total: 121ms	remaining: 0us
0:	learn: 1.2836059	total: 112ms	remaining: 0us
0:	learn: 1.3101268	total: 130ms	remaining: 0us
0:	learn: 1.2757136	total: 117ms	remain

__Задание 3. (2 балла)__

Проведите эксперименты:
1. Помогает ли предварительное понижение размерности с помощью PCA? 
2. Как зависит итоговое качество от n_features? Выходит ли оно на плато при росте n_features?
3. Важно ли, какую модель обучать — логистическую регрессию или SVM?

### Обучение линейной SVM модели с PCA и без

### Обучение логистической модели с PCA и без

### Бонус

__Задание 4. (Максимум 2 балла)__

Как вы, должно быть, помните с курса МО-1, многие алгоритмы машинного обучения работают лучше, если признаки данных некоррелированы. Оказывается, что для RFF существует модификация, позволяющая получать ортогональные случайные признаки (Orthogonal Random Features, ORF). Об этом методе можно прочитать в [статье](https://proceedings.neurips.cc/paper/2016/file/53adaf494dc89ef7196d73636eb2451b-Paper.pdf). Реализуйте класс для вычисления ORF по аналогии с основным заданием. Обратите внимание, что ваш класс должен уметь работать со случаем n_features > new_dim (в статье есть замечание на этот счет). Проведите эксперименты, сравнивающие RFF и ORF, сделайте выводы.

In [ ]:
# Your code here: (￣▽￣)/♫•*¨*•.¸¸♪

__Задание 5. (Максимум 2 балла)__

Поэкспериментируйте с функциями для вычисления новых случайных признаков. Не обязательно использовать косинус от скалярного произведения — можно брать знак от него, хэш и т.д. Придумайте побольше вариантов для генерации признаков и проверьте, не получается ли с их помощью добиваться более высокого качества. Также можете попробовать другой классификатор поверх случайных признаков, сравните результаты.

In [ ]:
# Your code here: (￣▽￣)/♫•*¨*•.¸¸♪